In [1]:
import os
import glob
import rasterio as rio
from rasterio.mask import raster_geometry_mask
from rasterio import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import pyproj
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform

In [2]:
# Functions

# Get panel masks

def get_panel_masks(geojson_path, ref_img_path):
    '''This function generates a mask that leaves only pixels inside 
    vine panels unmasked. 
    
    Inputs:
    geojson_path (str): path to a geojson file with the panel geometries. 
    Must be in same CRS as image, otherwise un-comment code section starting with "dst_crs".
    ref_img_path (str): path to the .tif file for the image
    
    Outputs:
    A list of masks, as Boolean arrays. True values are masked pixels.
    False values are unmasked pixels.
    '''
    with open(geojson_path, 'r') as f:
        panels = json.load(f)
        features = panels['features']
        features_ordered = sorted(features, key= lambda feature:(feature['properties']['row'],feature['properties']['panel'])) # ground truth order.
        panel_aois = [f['geometry'] for f in features_ordered]
    
    with rio.open(ref_img_path) as src:
        dst_crs = src.crs
        
        # geojson features (the field block geometries)
        # are often in WGS84
        # project these to the image coordinates
        wgs84 = pyproj.CRS('EPSG:4326')

        project = pyproj.Transformer.from_crs(wgs84, dst_crs, always_xy=True).transform
        proj_panel_aois = [transform(project, shape(p)) for p in panel_aois]

        masks = [raster_geometry_mask(src, [p], crop=False)[0]
                 for p in proj_panel_aois]                       
    return masks

# Get reflectance df

def get_rfl(img_path, masks):
    '''
    Function to extract per=panel reflectance values for 4-band SkySat images.
    Calculates the mean reflectance values for the pixels in each panel.
    The output is formatted as a pandas dataframe.
    
    Inputs:
    img_path - abs. path to the image       
    masks - a list of masks that correspond to the panel aois. 
    '''
    
    with rio.open(img_path,'r') as img:
        def _get_stats(mask, panel_number):
            panel_blue = np.ma.mean(np.ma.array(img.read(1), mask=mask))#/10000
            panel_green = np.ma.mean(np.ma.array(img.read(2), mask=mask))#/10000
            panel_red = np.ma.mean(np.ma.array(img.read(3), mask=mask))#/10000
            panel_nir = np.ma.mean(np.ma.array(img.read(4), mask=mask))#/10000
            veg_band= np.ma.mean(np.ma.array(img.read(5),mask=mask))
            savi_exp = np.ma.mean(np.ma.array(img.read(6),mask=mask))
            savi = np.ma.mean(np.ma.array(img.read(7),mask=mask))
            msavi = np.ma.mean(np.ma.array(img.read(8),mask=mask))
            evi = np.ma.mean(np.ma.array(img.read(9),mask=mask))
            arvi = np.ma.mean(np.ma.array(img.read(10),mask=mask))
            gri = np.ma.mean(np.ma.array(img.read(11),mask=mask))
            ndvi = np.ma.mean(np.ma.array(img.read(12),mask=mask))
            nir_blue = np.ma.mean(np.ma.array(img.read(13),mask=mask))
            nir_green = np.ma.mean(np.ma.array(img.read(14),mask=mask))
            red_blue = np.ma.mean(np.ma.array(img.read(15),mask=mask))
            green_red = np.ma.mean(np.ma.array(img.read(16),mask=mask))
            green_blue = np.ma.mean(np.ma.array(img.read(17),mask=mask))
            
            return {'acquired': img_path.split('/')[-1].split('_')[-8], # date from image filename
                    'plot': panel_number + 1,    #add one - panels are numbered starting at 1 in the 'real world'
                    'blue': panel_blue,
                    'green': panel_green,
                    'red': panel_red,
                    'nir': panel_nir,
                   'veg_smr': veg_band,
                    'savi_exp': savi_exp,
                    'savi': savi,
                    'msavi': msavi,
                    'evi': evi,
                    'arvi': arvi,
                    'gri': gri,
                    'ndvi': ndvi,
                    'nir_blue': nir_blue, 
                    'nir_green': nir_green,
                    'red_blue': red_blue,
                    'green_red':green_red,
                    'green_blue': green_blue
                   }
        
        data = [_get_stats(m, i) for i, m in enumerate(masks)]  # sets the parameters for the nested function as the mask (m) and corresponding panel number (i). 
        df = pd.DataFrame(data)
    
    return df

In [3]:
# image directory containing co-registered SkySat scenes 

img_dir = '../data/images/2022/SkySat/smr_vis_2022/'

# image pathnames
img_paths = glob.glob(img_dir+'*.tif')

# geojson with panel geometries

json_path = '../data/crit_panels_geojson.geojson'
json_path
img_paths

['../data/images/2022/SkySat/smr_vis_2022/SMR_allVIs_20220622_160829_ssc15d2_0014_analytic_SR_corg_clipped.tif',
 '../data/images/2022/SkySat/smr_vis_2022/SMR_allVIs_20220707_151445_ssc4d2_0006_analytic_SR_clipped_clipped.tif',
 '../data/images/2022/SkySat/smr_vis_2022/SMR_allVIs_20220720_153112_ssc4d2_0013_analytic_SR_clipped_clipped.tif']

In [4]:
img_paths[0].split('/')[-1].split('_')[-8]

'20220622'

In [5]:
# Get dataframes for all images in directory

master_df = pd.DataFrame()

for i in img_paths:
    masks = get_panel_masks(json_path, i)
    df = get_rfl(i, masks)
    master_df = pd.concat([master_df, df], axis=0)
    master_df = master_df.sort_values(by=['acquired', 'plot'])

master_df

,acquired,plot,blue,green,red,nir,veg_smr,savi_exp,savi,msavi,evi,arvi,gri,ndvi,nir_blue,nir_green,red_blue,green_red,green_blue
0,20220622,1,0.064216,0.096894,0.101359,0.370316,0.541185,0.422106,0.415183,0.293978,0.449359,0.456009,0.957620,0.570341,0.704397,0.585231,0.223934,-0.021887,0.203049
1,20220622,2,0.065437,0.097030,0.104597,0.358477,0.504905,0.396512,0.395456,0.266402,0.424668,0.428080,0.928531,0.548477,0.691283,0.574096,0.229761,-0.037173,0.194271
2,20220622,3,0.065977,0.097253,0.104310,0.355327,0.497562,0.392264,0.392434,0.261014,0.422489,0.427810,0.933402,0.546481,0.687012,0.570447,0.224910,-0.034609,0.191837
3,20220622,4,0.064136,0.096182,0.101911,0.361721,0.521191,0.408007,0.404437,0.276967,0.435540,0.443312,0.944762,0.560539,0.698788,0.579961,0.226893,-0.028564,0.199646
4,20220622,5,0.064762,0.095248,0.103503,0.353700,0.499395,0.392226,0.392062,0.259432,0.420352,0.426744,0.921422,0.547266,0.690398,0.575661,0.229728,-0.041130,0.190438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,20220720,316,0.033697,0.058660,0.056573,0.380930,0.753924,0.592149,0.518860,0.399414,0.552537,0.654849,1.042003,0.741183,0.837235,0.733009,0.254029,0.019332,0.271856
316,20220720,317,0.028686,0.053783,0.050524,0.372124,0.760645,0.602338,0.522732,0.393833,0.550595,0.674254,1.065356,0.760728,0.856704,0.747249,0.275992,0.031317,0.304679
317,20220720,318,0.029510,0.057121,0.053648,0.362910,0.726985,0.570949,0.506099,0.368590,0.528322,0.647113,1.066486,0.742465,0.849613,0.728016,0.290332,0.031610,0.319007
318,20220720,319,0.030410,0.057228,0.055586,0.366976,0.730182,0.571760,0.506095,0.373174,0.528638,0.638842,1.032497,0.736499,0.846694,0.729940,0.293914,0.015262,0.307716


In [6]:
# Append corresponding row and panel numbers for each plot 

panel_csv = pd.read_csv('../data/crit_panels.csv')
cols = panel_csv.iloc[:,1:] # adjust this according to the .csv
#cols
appended = pd.concat([master_df, cols],axis=1)
appended

,acquired,plot,blue,green,red,nir,veg_smr,savi_exp,savi,msavi,...,arvi,gri,ndvi,nir_blue,nir_green,red_blue,green_red,green_blue,row,panel
0,20220622,1,0.064216,0.096894,0.101359,0.370316,0.541185,0.422106,0.415183,0.293978,...,0.456009,0.957620,0.570341,0.704397,0.585231,0.223934,-0.021887,0.203049,1,1
1,20220622,2,0.065437,0.097030,0.104597,0.358477,0.504905,0.396512,0.395456,0.266402,...,0.428080,0.928531,0.548477,0.691283,0.574096,0.229761,-0.037173,0.194271,1,2
2,20220622,3,0.065977,0.097253,0.104310,0.355327,0.497562,0.392264,0.392434,0.261014,...,0.427810,0.933402,0.546481,0.687012,0.570447,0.224910,-0.034609,0.191837,1,3
3,20220622,4,0.064136,0.096182,0.101911,0.361721,0.521191,0.408007,0.404437,0.276967,...,0.443312,0.944762,0.560539,0.698788,0.579961,0.226893,-0.028564,0.199646,1,4
4,20220622,5,0.064762,0.095248,0.103503,0.353700,0.499395,0.392226,0.392062,0.259432,...,0.426744,0.921422,0.547266,0.690398,0.575661,0.229728,-0.041130,0.190438,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,20220720,316,0.033697,0.058660,0.056573,0.380930,0.753924,0.592149,0.518860,0.399414,...,0.654849,1.042003,0.741183,0.837235,0.733009,0.254029,0.019332,0.271856,20,12
316,20220720,317,0.028686,0.053783,0.050524,0.372124,0.760645,0.602338,0.522732,0.393833,...,0.674254,1.065356,0.760728,0.856704,0.747249,0.275992,0.031317,0.304679,20,13
317,20220720,318,0.029510,0.057121,0.053648,0.362910,0.726985,0.570949,0.506099,0.368590,...,0.647113,1.066486,0.742465,0.849613,0.728016,0.290332,0.031610,0.319007,20,14
318,20220720,319,0.030410,0.057228,0.055586,0.366976,0.730182,0.571760,0.506095,0.373174,...,0.638842,1.032497,0.736499,0.846694,0.729940,0.293914,0.015262,0.307716,20,15


In [7]:
appended.acquired.unique()


array(['20220622', '20220707', '20220720'], dtype=object)

In [8]:
scout_csv = pd.read_csv('../data/scout/scout_2022_gdf.csv')
scout_csv.Date.unique()

array(['2022-06-22', '2022-06-29', '2022-07-06', '2022-07-14',
       '2022-07-20', '2022-07-27', '2022-08-02', '2022-08-09',
       '2022-08-29'], dtype=object)

In [9]:
# Match scout data to image data

# Define filtering and matching parameters

first_row = 1
last_row = 20

match_dates = ['2022-06-22','2022-07-06', '2022-07-20']
match_cols =['row', 'panel', 'Date']


# Filter and join datasets

img_filtered = appended[appended['row']>=first_row]
img_filtered = img_filtered[img_filtered['row']<=last_row]

scout_filtered = scout_csv[scout_csv['Row']>=first_row]
scout_filtered = scout_csv[scout_csv['Row']<=last_row]

scout_filtered = scout_filtered[scout_filtered['Date'].isin(match_dates)]
scout_filtered = scout_filtered.rename(columns={'Row':'row', 'Panel':'panel'})
scout_filtered[["row", "panel"]] = scout_filtered[["row", "panel"]].apply(pd.to_numeric)

# Create a 'Date' column in the image dataframe to match the scout date
def match_days(row):
    if row['acquired']=='20220622':
        return '2022-06-22'
    if row['acquired']=='20220707':
        return '2022-07-06'
    if row['acquired']=='20220720':
        return '2022-07-20'

img_filtered['Date'] = img_filtered.apply(lambda row: match_days(row), axis=1)

# Join and drop NANs + unneeded columns

joined = img_filtered.join(scout_filtered.set_index(match_cols), on=match_cols)
cleaned = joined.dropna(axis=0,subset=['DM_severity'])

cleaned.info()
# Uncomment below for masked rasters
#no_nullpix = cleaned.dropna(axis=0,subset=['blue', 'green', 'red', 'nir'])

#no_nullpix.info()
#joined = joined.rename(columns={'mean_sev_top_bot':'DM_severity'})
#cleaned = joined.dropna(axis=0,subset=['geometry', 'Treatment', 'leaf_avg', 'centroid'])
#cleaned = cleaned.drop(['Disease'], axis=1)




<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 0 to 318
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acquired     720 non-null    object 
 1   plot         720 non-null    int64  
 2   blue         720 non-null    float64
 3   green        720 non-null    float64
 4   red          720 non-null    float64
 5   nir          720 non-null    float64
 6   veg_smr      720 non-null    float64
 7   savi_exp     720 non-null    float64
 8   savi         720 non-null    float64
 9   msavi        720 non-null    float64
 10  evi          720 non-null    float64
 11  arvi         720 non-null    float64
 12  gri          720 non-null    float64
 13  ndvi         720 non-null    float64
 14  nir_blue     720 non-null    float64
 15  nir_green    720 non-null    float64
 16  red_blue     720 non-null    float64
 17  green_red    720 non-null    float64
 18  green_blue   720 non-null    float64
 19  row     

In [14]:
# Export dataframe as .csv

output_fp = '../data/img_scout_dfs/2022/'

#cleaned.to_csv(output_fp+'smr_vis_skysat_scout_2022.csv', index=False)

In [10]:
# Merge incidence data

inc = pd.read_csv('/Users/kathleenkanaley/Desktop/grapes_from_space/data/scout/scout_incidence/scout_inc_2022.csv')

inc.rename(columns={"Row": "row", "Panel": "panel"}, inplace=True)
inc.columns

Index(['Date', 'row', 'panel', 'Treatment', 'Block', 'PM_severity',
       'DM_severity', 'total_dis', 'geometry', 'centroid', 'DM_inc', 'PM_inc'],
      dtype='object')

In [11]:
merged_2022 = cleaned.merge(inc, how='left', on=['Date', 
                                               'row', 
                                               'panel',
                                               'Treatment', 
                                               'Block', 
                                               'PM_severity','DM_severity',
                                              'total_dis','geometry','centroid'])
merged_2022.shape


(720, 31)

In [12]:
cleaned.shape

(720, 29)

In [13]:
merged_2022.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 0 to 719
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acquired     720 non-null    object 
 1   plot         720 non-null    int64  
 2   blue         720 non-null    float64
 3   green        720 non-null    float64
 4   red          720 non-null    float64
 5   nir          720 non-null    float64
 6   veg_smr      720 non-null    float64
 7   savi_exp     720 non-null    float64
 8   savi         720 non-null    float64
 9   msavi        720 non-null    float64
 10  evi          720 non-null    float64
 11  arvi         720 non-null    float64
 12  gri          720 non-null    float64
 13  ndvi         720 non-null    float64
 14  nir_blue     720 non-null    float64
 15  nir_green    720 non-null    float64
 16  red_blue     720 non-null    float64
 17  green_red    720 non-null    float64
 18  green_blue   720 non-null    float64
 19  row     

In [14]:
output_fp = '/Users/kathleenkanaley/Desktop/grapes_from_space/data/img_scout_dfs/'
# merged_2022.to_csv(output_fp+'2022/smr_vis_skysat_scout_2022.csv', index=False)